In [ ]:
import os
import re
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Mount Google Drive. Confirm access permissions to permit this notebook to access your Google Drive files
from google.colab import drive
drive.mount('/content/drive')
# drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
df_loan = pd.read_csv('/content/drive/MyDrive/datasci-210-project/fraud-detect/data/ppp-data-combined.csv')

<ipython-input-3-58cf10514e43>:1: DtypeWarning: Columns (17,51) have mixed types. Specify dtype option on import or set low_memory=False.
  df_loan = pd.read_csv('/content/drive/MyDrive/datasci-210-project/fraud-detect/data/ppp-data-combined.csv')


In [ ]:
df_loan.head()

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate
0,0,2593527208,04/16/2020,1094.0,PPP,"LORI L. HOLDER, LCSW",504 MAIN ST Suite 472,LEWISTON,ID,83501-1806,...,Sole Proprietorship,59698.0,Umpqua Bank,ROSEBURG,OR,Unanswered,Unanswered,NaN,503.92,02/12/2021
1,1,2741007306,04/29/2020,1087.0,PPP,THUNDER HILL FITNESS CENTER LLC,283 S. Middlefork Rd,GARDEN VALLEY,ID,83622,...,Limited Liability Partnership,106925.0,Mountain America FCU,SANDY,UT,Unanswered,Unanswered,NaN,505.25,05/25/2021
2,2,3342508906,04/28/2021,1087.0,PPP,KATRINA RAMIREZ,2877 N Bliss Dr,Idaho Falls,ID,83401-5873,...,Self-Employed Individuals,317954.0,"Newtek Small Business Finance, Inc.",LAKE SUCCESS,NY,Unanswered,Unanswered,NaN,502.43,10/27/2021
3,3,3379798208,08/04/2020,1087.0,PPP,JAMES PARIS SCHOOL OF MUSIC,3048 W SAN REMO ST,MERIDIAN,ID,83646-3214,...,Sole Proprietorship,96211.0,Idaho Central CU,CHUBBUCK,ID,Male Owned,Unanswered,NaN,502.16,01/12/2021
4,4,3753429003,05/20/2021,1087.0,PPP,ANGEL RICE,4088 S Riva Ridge Way,Boise,ID,83709-4448,...,Independent Contractors,188361.0,"Prestamos CDFI, LLC",Phoenix,AZ,Female Owned,Non-Veteran,NaN,503.40,02/03/2022


In [ ]:
column_names = df_loan.columns.tolist()
column_names

['Unnamed: 0',
 'LoanNumber',
 'DateApproved',
 'SBAOfficeCode',
 'ProcessingMethod',
 'BorrowerName',
 'BorrowerAddress',
 'BorrowerCity',
 'BorrowerState',
 'BorrowerZip',
 'LoanStatusDate',
 'LoanStatus',
 'Term',
 'SBAGuarantyPercentage',
 'InitialApprovalAmount',
 'CurrentApprovalAmount',
 'UndisbursedAmount',
 'FranchiseName',
 'ServicingLenderLocationID',
 'ServicingLenderName',
 'ServicingLenderAddress',
 'ServicingLenderCity',
 'ServicingLenderState',
 'ServicingLenderZip',
 'RuralUrbanIndicator',
 'HubzoneIndicator',
 'LMIIndicator',
 'BusinessAgeDescription',
 'ProjectCity',
 'ProjectCountyName',
 'ProjectState',
 'ProjectZip',
 'CD',
 'JobsReported',
 'NAICSCode',
 'Race',
 'Ethnicity',
 'UTILITIES_PROCEED',
 'PAYROLL_PROCEED',
 'MORTGAGE_INTEREST_PROCEED',
 'RENT_PROCEED',
 'REFINANCE_EIDL_PROCEED',
 'HEALTH_CARE_PROCEED',
 'DEBT_INTEREST_PROCEED',
 'BusinessType',
 'OriginatingLenderLocationID',
 'OriginatingLender',
 'OriginatingLenderCity',
 'OriginatingLenderState',
 '

In [ ]:
df_loan['NAICSCode_2digits'] = df_loan['NAICSCode'].astype(str).str[:2]

In [ ]:
df_loan['NAICSCode_2digits'].unique()

array(['62', '71', '42', '81', '49', '54', '56', '44', '23', '45', '52',
       '11', '32', '48', '92', '61', '72', '33', '51', '99', '53', 'na',
       '31', '55', '21', '22'], dtype=object)

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/datasci-210-project/US_Census_Data/US_Census_data.xlsx')
census_grouped = df.groupby(['State_AB','NAICS Code'])['Annual Payroll ($1,000)'].mean().reset_index()

In [ ]:
census_grouped['NAICS Code'] = census_grouped['NAICS Code'].astype(str)

In [ ]:
merged_df = df_loan.merge(census_grouped, left_on = ['BorrowerState','NAICSCode_2digits'], right_on = ['State_AB','NAICS Code'],how = 'left')
merged_df

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,NAICSCode_2digits,State_AB,NAICS Code,"Annual Payroll ($1,000)"
0,0,2593527208,04/16/2020,1094.0,PPP,"LORI L. HOLDER, LCSW",504 MAIN ST Suite 472,LEWISTON,ID,83501-1806,...,OR,Unanswered,Unanswered,NaN,503.92,02/12/2021,62,ID,62,2.573329e+06
1,1,2741007306,04/29/2020,1087.0,PPP,THUNDER HILL FITNESS CENTER LLC,283 S. Middlefork Rd,GARDEN VALLEY,ID,83622,...,UT,Unanswered,Unanswered,NaN,505.25,05/25/2021,71,ID,71,1.052645e+05
2,2,3342508906,04/28/2021,1087.0,PPP,KATRINA RAMIREZ,2877 N Bliss Dr,Idaho Falls,ID,83401-5873,...,NY,Unanswered,Unanswered,NaN,502.43,10/27/2021,42,ID,42,1.108028e+06
3,3,3379798208,08/04/2020,1087.0,PPP,JAMES PARIS SCHOOL OF MUSIC,3048 W SAN REMO ST,MERIDIAN,ID,83646-3214,...,ID,Male Owned,Unanswered,NaN,502.16,01/12/2021,81,ID,81,3.478393e+05
4,4,3753429003,05/20/2021,1087.0,PPP,ANGEL RICE,4088 S Riva Ridge Way,Boise,ID,83709-4448,...,AZ,Female Owned,Non-Veteran,NaN,503.40,02/03/2022,49,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11765547,11765547,8355007904,06/18/2020,1087.0,PPP,BROOKE OBENCHAN SYLIST,1103 N MAIN ST,MERIDIAN,ID,83642-2305,...,SD,Unanswered,Unanswered,NaN,509.69,01/04/2022,81,ID,81,3.478393e+05
11765548,11765548,1561987806,05/21/2020,1087.0,PPP,PARK AVENUE FINANCE LLC,329 S WOODRUFF AVE,IDAHO FALLS,ID,83401-4322,...,UT,Unanswered,Unanswered,NaN,504.52,04/19/2021,81,ID,81,3.478393e+05
11765549,11765549,1742097308,04/28/2020,1087.0,PPP,ASLETT & ASLETT WEALTH MANAGEMENT,10 N. Amaya Way,Nampa,ID,83651,...,ID,Female Owned,Unanswered,NaN,503.34,01/08/2021,54,ID,54,1.288226e+06
11765550,11765550,2188397409,05/05/2020,1087.0,PPP,IDAHO ART LAB,2355 S Yellowstone Highway,SAINT ANTHONY,ID,83445-5746,...,ID,Unanswered,Unanswered,NaN,504.67,04/22/2021,61,ID,61,2.005263e+05


In [ ]:
merged_data = merged_df.drop(['State_AB','NAICS Code'],axis = 1)
merged_data.rename(columns={'Annual Payroll ($1,000)': 'Census Annual Payroll ($1,000)'},inplace= True)
merged_data

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,NAICSCode_2digits,"Census Annual Payroll ($1,000)"
0,0,2593527208,04/16/2020,1094.0,PPP,"LORI L. HOLDER, LCSW",504 MAIN ST Suite 472,LEWISTON,ID,83501-1806,...,Umpqua Bank,ROSEBURG,OR,Unanswered,Unanswered,NaN,503.92,02/12/2021,62,2.573329e+06
1,1,2741007306,04/29/2020,1087.0,PPP,THUNDER HILL FITNESS CENTER LLC,283 S. Middlefork Rd,GARDEN VALLEY,ID,83622,...,Mountain America FCU,SANDY,UT,Unanswered,Unanswered,NaN,505.25,05/25/2021,71,1.052645e+05
2,2,3342508906,04/28/2021,1087.0,PPP,KATRINA RAMIREZ,2877 N Bliss Dr,Idaho Falls,ID,83401-5873,...,"Newtek Small Business Finance, Inc.",LAKE SUCCESS,NY,Unanswered,Unanswered,NaN,502.43,10/27/2021,42,1.108028e+06
3,3,3379798208,08/04/2020,1087.0,PPP,JAMES PARIS SCHOOL OF MUSIC,3048 W SAN REMO ST,MERIDIAN,ID,83646-3214,...,Idaho Central CU,CHUBBUCK,ID,Male Owned,Unanswered,NaN,502.16,01/12/2021,81,3.478393e+05
4,4,3753429003,05/20/2021,1087.0,PPP,ANGEL RICE,4088 S Riva Ridge Way,Boise,ID,83709-4448,...,"Prestamos CDFI, LLC",Phoenix,AZ,Female Owned,Non-Veteran,NaN,503.40,02/03/2022,49,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11765547,11765547,8355007904,06/18/2020,1087.0,PPP,BROOKE OBENCHAN SYLIST,1103 N MAIN ST,MERIDIAN,ID,83642-2305,...,"Wells Fargo Bank, National Association",SIOUX FALLS,SD,Unanswered,Unanswered,NaN,509.69,01/04/2022,81,3.478393e+05
11765548,11765548,1561987806,05/21/2020,1087.0,PPP,PARK AVENUE FINANCE LLC,329 S WOODRUFF AVE,IDAHO FALLS,ID,83401-4322,...,"Zions Bank, A Division of",SALT LAKE CITY,UT,Unanswered,Unanswered,NaN,504.52,04/19/2021,81,3.478393e+05
11765549,11765549,1742097308,04/28/2020,1087.0,PPP,ASLETT & ASLETT WEALTH MANAGEMENT,10 N. Amaya Way,Nampa,ID,83651,...,Idaho Central CU,CHUBBUCK,ID,Female Owned,Unanswered,NaN,503.34,01/08/2021,54,1.288226e+06
11765550,11765550,2188397409,05/05/2020,1087.0,PPP,IDAHO ART LAB,2355 S Yellowstone Highway,SAINT ANTHONY,ID,83445-5746,...,Bank of Idaho,IDAHO FALLS,ID,Unanswered,Unanswered,NaN,504.67,04/22/2021,61,2.005263e+05
